In [5]:
import yaml
from glob import glob

# Initialize a list to store the collected data
data = {
    "s-different-prob": [],
    "l-different-prob": [],
    "xl-different-prob": [],
    "xxl-different-prob": [],
}


# Function to check if a config already exists in the data list
def config_exists(data_list, config):
    for item in data_list:
        if (
            item["room_size"] == config["room_size"]
            and item["qa_policy"] == config["qa_policy"]
            and item["explore_policy"] == config["explore_policy"]
            and item["mm_policy"] == config["mm_policy"]
            and item["memory_size"] == config["memory_size"]
        ):
            return True
    return False


# Iterate through all 'results.yaml' files in the specified directory
for path in glob("./training-results/*/results.yaml"):
    with open(path, "r") as f:
        results = yaml.safe_load(f)

    # Extract test score mean and standard deviation
    test_mean = results["test_score"]["mean"]
    test_std = results["test_score"]["std"]

    # Open the corresponding 'train.yaml' file to extract additional data
    with open(path.replace("results.yaml", "train.yaml")) as f:
        hp = yaml.safe_load(f)

    # Extract relevant data from 'train.yaml'
    room_size = hp["env_config"]["room_size"]
    qa_policy = hp["qa_policy"]
    explore_policy = hp["explore_policy"]
    mm_policy = hp.get("mm_policy", None)  # Use get to handle missing keys
    memory_size = hp.get(
        "max_long_term_memory_size", None
    )  # Use get to handle missing keys

    config = {
        "test_mean": test_mean,
        "test_std": test_std,
        "room_size": room_size,
        "qa_policy": qa_policy,
        "explore_policy": explore_policy,
        "mm_policy": mm_policy,
        "memory_size": memory_size,
    }

    # Check if the config already exists before appending
    if not config_exists(data[room_size], config):
        data[room_size].append(config)
    else:
        print(f"Config already exists for room size: {room_size}, skipping...")

Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...
Config already exists for room size: l-different-prob, skipping...


In [6]:
import pandas as pd

# Create a Pandas DataFrame from the collected data
df = pd.DataFrame(data["l-different-prob"])


# Sort the DataFrame by 'test_mean' in descending order for better readability
df_sorted = df.sort_values(by="test_mean", ascending=False)

# Display the sorted DataFrame
df_sorted

,test_mean,test_std,room_size,qa_policy,explore_policy,mm_policy,memory_size
12,792.0,0.00,l-different-prob,latest,bfs,FIFO,1024.0
21,746.9,56.89,l-different-prob,latest,bfs,remove_oldest,1024.0
26,724.9,52.29,l-different-prob,latest,dijkstra,remove_oldest,512.0
22,720.0,27.47,l-different-prob,latest,dijkstra,remove_oldest,1024.0
27,717.5,63.38,l-different-prob,latest,bfs,remove_oldest,512.0
17,699.7,40.81,l-different-prob,latest,bfs,remove_oldest,256.0
8,692.5,47.41,l-different-prob,latest,dijkstra,remove_oldest,128.0
14,690.9,57.63,l-different-prob,latest,dijkstra,remove_oldest,256.0
15,651.2,39.46,l-different-prob,latest,bfs,remove_oldest,128.0
18,475.1,50.28,l-different-prob,latest,dijkstra,remove_oldest,64.0


In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the collected data
df = pd.DataFrame(data["xl-different-prob"])


# Sort the DataFrame by 'test_mean' in descending order for better readability
df_sorted = df.sort_values(by="test_mean", ascending=False)

# Display the sorted DataFrame
df_sorted

In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the collected data
df = pd.DataFrame(data["xxl-different-prob"])


# Sort the DataFrame by 'test_mean' in descending order for better readability
df_sorted = df.sort_values(by="test_mean", ascending=False)

# Display the sorted DataFrame
df_sorted

In [ ]:
import shutil
from copy import deepcopy
from datetime import datetime, timedelta
import random
from typing import Any

import numpy as np
import gymnasium as gym
from rdflib import XSD, Graph, Literal, Namespace, URIRef

from humemai.rdflib import Humemai

from rdflib import BNode, Graph, Literal, Namespace, URIRef
import logging
from agent import ShortTermAgent, LongTermAgent

ns = Namespace("https://humem.ai/ontology#")

# Disable logging
logging.getLogger().setLevel(logging.CRITICAL)

for qa_policy in ["one_hop"]:
    for explore_policy in ["avoid_walls"]:
        print(f"QA policy: {qa_policy}, Explore policy: {explore_policy}")
        agent = ShortTermAgent(
            env_config={
                "question_prob": 1.0,
                "seed": 0,
                "terminates_at": 99,
                "randomize_observations": "all",
                "room_size": "xxl-different-prob",
                "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                "make_everything_static": False,
                "num_total_questions": 1000,
                "question_interval": 1,
                "include_walls_in_observations": True,
                "deterministic_objects": False,
            },
            qa_policy=qa_policy,
            explore_policy=explore_policy,
            num_samples_for_results=10,
        )

        agent.test()

In [ ]:
agent.humemai.get_raw_triple_count(), agent.humemai.get_main_triple_count(), agent.humemai.get_memory_count()

In [ ]:
agent.humemai.get_short_term_memory_count(), agent.humemai.get_long_term_memory_count(), agent.humemai.get_main_triple_count(), agent.humemai.get_raw_triple_count()

In [ ]:
agent.humemai.print_raw_triples()

In [ ]:
agent.humemai.print_main_triples()

In [2]:
agent.humemai.print_memories()

(room_000, east, room_001, {'memoryID': '5', 'recalled': '0', 'event_time': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(room_000, west, wall, {'memoryID': '6', 'recalled': '0', 'event_time': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(room_000, north, wall, {'memoryID': '7', 'recalled': '0', 'event_time': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(room_000, south, wall, {'memoryID': '8', 'recalled': '0', 'event_time': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(agent, at_location, room_000, {'memoryID': '9', 'recalled': '0', 'event_time': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(dep_003, at_location, room_001, {'memoryID': '16', 'recalled': '12', 'event_time': '2025-01-02T00:00:00', 'last_accessed': '2025-01-03T00:00:00'})
(room_001, west, room_000, {'memoryID': '17', 'recalled': '0', 'event_time': '2025-01-02T00:00:00', 'last_accessed': '2025-01-02T00:00:00'})
(room_001, north,

In [3]:
from rdflib import BNode, Graph, Literal, Namespace, URIRef
import logging
from agent import ShortTermAgent, LongTermAgent

# Disable logging
logging.getLogger().setLevel(logging.CRITICAL)

for room_size in ["l-different-prob"]:
    for qa_policy in ["latest"]:
        for explore_policy in ["dijkstra", "bfs"]:
            for mm_policy in ["FIFO"]:
                for max_long_term_memory_size in [
                    # 0,
                    # 2,
                    # 4,
                    # 8,
                    # 16,
                    # 32,
                    64,
                    # 128,
                    # 256,
                    # 512,
                    # 1024,
                ]:

                    print(
                        f"QA policy: {qa_policy}, Explore policy: {explore_policy}, "
                        f"MM policy: {mm_policy}, "
                        f"Max long term memory size: {max_long_term_memory_size}"
                    )
                    agent = LongTermAgent(
                        env_config={
                            "question_prob": 1.0,
                            "seed": 0,
                            "terminates_at": 9,
                            "randomize_observations": "all",
                            "room_size": room_size,
                            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                            "make_everything_static": False,
                            "num_total_questions": 1000,
                            "question_interval": 1,
                            "include_walls_in_observations": True,
                            "deterministic_objects": False,
                        },
                        qa_policy=qa_policy,
                        explore_policy=explore_policy,
                        mm_policy=mm_policy,
                        max_long_term_memory_size=max_long_term_memory_size,
                        num_samples_for_results=1,
                    )

                    agent.test()

QA policy: latest, Explore policy: dijkstra, MM policy: FIFO, Max long term memory size: 64
Episode 1/1 completed. Score: 444.00
Results: {'test_score': {'mean': 444.0, 'std': 0.0, 'min': 444.0, 'max': 444.0}, 'num_episodes': 1}
QA policy: latest, Explore policy: bfs, MM policy: FIFO, Max long term memory size: 64
Episode 1/1 completed. Score: 444.00
Results: {'test_score': {'mean': 444.0, 'std': 0.0, 'min': 444.0, 'max': 444.0}, 'num_episodes': 1}


In [ ]:
foo = agent.humemai.graph.query(
    """
PREFIX humemai: <https://humem.ai/ontology#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (SUM(?recalledValue) AS ?totalRecalled)
WHERE {
  ?statement rdf:type rdf:Statement ;
             humemai:recalled ?recalledValue .
}
"""
)

In [ ]:
# SPARQLResult can be iterated as if it were a list of rows
for row in foo:
    print(row)
    # row.totalRecalled corresponds to the variable ?totalRecalled
    total_recalled_value = row.totalRecalled
    print(f"Sum of all recalled values: {total_recalled_value}")

In [ ]:
agent.humemai.get_long_term_memory_count()